# Example for:
Transfer Learning Empirical Experiment from all 10 classes of FashionMNIST into all 10 classes of MNIST 

### Setup and Hyperparams

In [1]:
# Specify which gpu
import os
gpu_id = 1
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

import sys
sys.path.append('/mnt/c/Users/Arnisa/Desktop/MP/mp-tl-study')
from functions.utils import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)  # if using multi-GPU

/home/arnisa/anaconda3/envs/mp-env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# cuts=0 means: end-to-end model if we are reinitializing
cuts = [0,1,2,3,4,5]

In [3]:
# Changes Here for the experiments
params = {
      # MODEL ARCHITECTURE PARAMS
      'depth': 6,
      'num_channels': 64, # num channels for CNN
      'activation_function': nn.ReLU,
      'kernel_size': 3,
      # TRAINING PARAMS
      'device': device,
      'lr_pretrain': 0.001,   
      'lr_fine_tune': 0.001,  #
      'num_train': 40,
      'early_stop_patience': 6,
      'save_best': False,
      'save_checkpoints': False,
      'is_cnn': True,
      'is_debug': False,
      'classification_report_flag': False,
      'batch_size':64,
      # DATASET PARAMS
      'pre_train_classes': list(range(10)),
      'fine_tune_classes': list(range(10)),
      'val_split': 0.1,
      'num_workers': 0,
      'generate_dataset_seed': 42,
      # EXPERIMENT SETTING PARAMS
      'use_pooling': True,  
      'pooling_every_n_layers': 2, # add pooling after every n layers specified here. For only one pooling after all the CNN layers, this equals params['depth']
      'pooling_stride': 2,
      'freeze': True,         # VARIABLE
      'reinit': True,         # VARIABLE
      'truncate': False,      # VARIABLE
    }

In [4]:
root_dir = './data'  # Specify your data directory here
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataloader_wrapped = TransferLearningWrapper(params, datasets.FashionMNIST, datasets.MNIST, root_dir, transform=transform)

## Pretraining

In [5]:
pretrained_model = CustomCNN(params, dataloader_wrapped.output_dim)
pretrained_model.to(device)

CustomCNN(
  (conv0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act0): ReLU()
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): ReLU()
  (pool3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act4): ReLU()
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act5): ReLU()
  (pool5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=576, out_features=10, bias=True)
)

In [13]:
# Train and evaluate - Skip if loading saved model!
trainer = Trainer(pretrained_model, dataloader_wrapped, params["lr_pretrain"], params)
train_acc, test_acc, effective_epochs, checkpoints = trainer.train()

print(f"Final Training Accuracy: {train_acc:.4f}")
print(f"Final Test Accuracy: {test_acc:.4f}")

Epoch: 0 	Training Accuracy: 85.60%
Validation Accuracy: 83.24%
Epoch: 1 	Training Accuracy: 89.10%
Validation Accuracy: 86.24%
Epoch: 2 	Training Accuracy: 90.89%
Validation Accuracy: 87.55%
Epoch: 3 	Training Accuracy: 92.12%
Validation Accuracy: 88.14%
Epoch: 4 	Training Accuracy: 92.62%
Validation Accuracy: 88.64%
Epoch: 5 	Training Accuracy: 92.89%
Validation Accuracy: 88.62%
Epoch: 6 	Training Accuracy: 93.30%
Validation Accuracy: 88.86%
Epoch: 7 	Training Accuracy: 93.62%
Validation Accuracy: 89.02%
Epoch: 8 	Training Accuracy: 93.84%
Validation Accuracy: 89.12%
Epoch: 9 	Training Accuracy: 93.81%
Validation Accuracy: 88.62%
Epoch: 10 	Training Accuracy: 94.22%
Validation Accuracy: 89.02%
Epoch: 11 	Training Accuracy: 94.16%
Validation Accuracy: 89.19%
Epoch: 12 	Training Accuracy: 94.35%
Validation Accuracy: 88.81%
Epoch: 13 	Training Accuracy: 94.43%
Validation Accuracy: 88.98%
Epoch: 14 	Training Accuracy: 94.51%
Validation Accuracy: 88.86%
Epoch: 15 	Training Accuracy: 94.67

In [6]:
pretrained_model.load_state_dict(torch.load('pretrained_models/regular_classes.pth', map_location=device))

<All keys matched successfully>

In [7]:
eval(pretrained_model, device, dataloader_wrapped.val_loader, debug=True, classification_report_flag=True, is_cnn=True)


Average loss: 0.1498, Accuracy: 5668.0/6000 (94%)

              precision    recall  f1-score   support

     Class 0       0.88      0.92      0.90       658
     Class 1       1.00      1.00      1.00       611
     Class 2       0.94      0.90      0.92       612
     Class 3       0.95      0.94      0.95       562
     Class 4       0.93      0.93      0.93       607
     Class 5       1.00      0.99      1.00       613
     Class 6       0.82      0.81      0.82       600
     Class 7       0.97      0.99      0.98       537
     Class 8       0.99      0.99      0.99       598
     Class 9       0.99      0.98      0.98       602

    accuracy                           0.94      6000
   macro avg       0.95      0.95      0.95      6000
weighted avg       0.94      0.94      0.94      6000



0.9446666666666667

In [16]:
# save model for later use
torch.save(pretrained_model.state_dict(), 'pretrained_models/regular_classes.pth')

## Fine-tuning Experiments

### Baselines
We also reuse the baselines a lot! so skip if we already have the jsons

In [8]:
baselines_results = []
percentages = [0.001, 0.01, 0.1, 0.5, 1.0]

In [ ]:
dataloader_wrapped.update_phase('finetune')

for sampled_percentage in percentages:      
    if sampled_percentage <= 0.01:
        repeats = 25
    elif sampled_percentage < 0.5:
        repeats = 20
    else:
        repeats = 5
    
    for repeat in range(repeats):
        print(f"\nSampled Percentage: {sampled_percentage}, Lr: {params['lr_fine_tune']}, Repeat: {repeat}")

        # Reduce the dataset
        train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed = repeat)
        torch.manual_seed(repeat)
        dataset_namespace_new = SimpleNamespace(train_loader=train_loader_reduced, test_loader=dataloader_wrapped.test_loader, val_loader=dataloader_wrapped.val_loader)

        # Copy and then cut the model - we already deepcopy it in the function: pretrained_model
        params_tmp = copy.deepcopy(params)
        params_tmp["reinit"] = True
        model_new = cut_custom_cnn_model(pretrained_model, cut_point=0, params=params_tmp, output_dim=dataloader_wrapped.output_dim)
        model_new.to(device)

        # Train and evaluate
        trainer = Trainer(model_new, dataset_namespace_new, params['lr_fine_tune'], params)
        train_acc, test_acc, effective_epochs, checkpoints = trainer.train(verbose=0)
        print(f"Training Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

        # Store the results
        baselines_results.append({"lr":params['lr_fine_tune'], "sampled_percentage":sampled_percentage, "sampled_cut_point":-1, "repeat":repeat, "train_acc":train_acc, "test_acc":test_acc}) # -1 for the cut point means it's baseline

In [21]:
# save baseline results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + baselines_results

with open(f'results/baselines_regular_classes.json', 'w') as f:
    json.dump(results, f)

### Fine-tuning

In [10]:
results = []
percentages = [0.001, 0.01, 0.1, 0.5, 1.0]

In [ ]:
dataloader_wrapped.update_phase('finetune')

for sampled_percentage in percentages:

    if sampled_percentage <= 0.01:
        repeats = 25
    elif sampled_percentage < 0.5:
        repeats = 20
    else:
        repeats = 5
        
    for sampled_cut_point in cuts:

        for repeat in range(repeats):
            print(f"\nSampled Percentage: {sampled_percentage}, Sampled Cut Point: {sampled_cut_point}, Lr: {params['lr_fine_tune']}, Repeat: {repeat}")

            # Reduce the dataset
            train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed=repeat)
            dataset_namespace_new = SimpleNamespace(train_loader=train_loader_reduced, test_loader=dataloader_wrapped.test_loader, val_loader=dataloader_wrapped.val_loader)
            torch.manual_seed(repeat) # because in the cut function we reinitialize some layers too (at least the dense layers)
            
            # Copy and then cut the model - we already deepcopy it in the function: pretrained_model
            model_new = cut_custom_cnn_model(pretrained_model, cut_point=sampled_cut_point, params=params, output_dim=dataloader_wrapped.output_dim)
            model_new.to(device)
            
            # Train and evaluate
            trainer = Trainer(model_new, dataset_namespace_new, params['lr_fine_tune'], params)
            train_acc, test_acc, effective_epochs, checkpoints = trainer.train(verbose=0)
            print(f"Training Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

            # Store the results
            results.append({"lr":params['lr_fine_tune'], "sampled_percentage":sampled_percentage, "sampled_cut_point":sampled_cut_point, "repeat":repeat, "train_acc":train_acc, "test_acc":test_acc})

In [27]:
# save fine-tuning results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + results

with open(f'results/cuts_regular_classes_FR.json', 'w') as f:
    json.dump(results, f)